In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.contrib.tensorboard.plugins import projector
import os
import numpy as np

In [2]:
# 载入数据
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
tf.reset_default_graph()
# 图片数量
image_cols = 60
image_rows = 60
# 定义会话
sess = tf.Session()

image_num = image_cols * image_rows

# 载入图片，把一定数量的测试图片载入内存
stack_pre_data = mnist.test.images[:image_num]
stack_data = tf.stack(stack_pre_data)

embedding = tf.Variable(stack_data, trainable=False, name='embedding')
       
x = tf.placeholder(tf.float32,[None,784],name='x-input')
y = tf.placeholder(tf.float32,[None,10],name='y-input')
    
# 显示图片
with tf.name_scope('input_reshape'):
    image_shaped_input = tf.reshape(x,[-1,28,28,1])
    tf.summary.image('Input', image_shaped_input,10) # 放进去10张图片
    
with tf.name_scope('model'):
    hidden1 = tf.layers.dense(x, 128, activation=tf.nn.relu, name='hidden-1')
    hidden2 = tf.layers.dense(hidden1, 32, activation=tf.nn.relu, name='hidden-2')
    logits = tf.layers.dense(hidden2, 10, activation=None, name='logits')
    prediction = tf.nn.softmax(logits)        
        
with tf.name_scope('metries'):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=logits))
    tf.summary.scalar('Loss',loss)
    train_step = tf.train.GradientDescentOptimizer(0.2).minimize(loss,name="train")
    correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(prediction,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
    tf.summary.scalar('Accuracy',accuracy)   

    
#合并所有summary
merged = tf.summary.merge_all()
#使用Projector必须要有saver
saver = tf.train.Saver()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [4]:
PROJECTOR_DIR = "./projector/"
PROJECTOR_META_FILENAME = "metadata.tsv"
PROJECTOR_META_PATH =os.path.join(PROJECTOR_DIR,PROJECTOR_META_FILENAME)
if not os.path.exists(PROJECTOR_META_PATH):
    test_images_labels = np.argmax(mnist.test.labels[:image_num],1)
    os.makedirs(PROJECTOR_DIR,exist_ok=True)
    with open(PROJECTOR_META_PATH,'w') as f:
        for i in range(image_num):
            f.write(str(test_images_labels[i]) + '\n')

In [7]:
from PIL import Image
#导出sprite.png
SPRITE_PNG_IMAGE = "sprites_image.png"
SPRITE_PNG_PATH = os.path.join(PROJECTOR_DIR,SPRITE_PNG_IMAGE)

if not os.path.exists(SPRITE_PNG_PATH):
    sprite_image_data = []
    for row in np.arange(image_rows):
        col_datas = []
        for col in np.arange(image_cols):
            data_index = col + row*image_cols
            col_data = stack_pre_data[data_index]
            col_data = np.reshape(col_data,(28,28))
            col_datas.append(col_data)
        row_data = np.concatenate(col_datas,axis=1)
        sprite_image_data.append(row_data)
    sprite_image_data = np.concatenate(sprite_image_data,axis=0)
    sprite_image_data = 0.001 - sprite_image_data
    #mode_list = ['1', 'L', 'I', 'F', 'P', 'RGB', 'RGBA', 'CMYK', 'YCbCr' ]
    sprite_image = Image.fromarray(sprite_image_data,"I")   #I表示一个像素一个int值

    sprite_image.save(SPRITE_PNG_PATH,"png") 
    print("width = {} pixels, height={} pixels,  rows={},  cols={}".format(sprite_image.width,sprite_image.height,sprite_image.height/28,sprite_image.width/28))

width = 1680 pixels, height=1680 pixels,  rows=60.0,  cols=60.0


In [6]:
batch_size = 1024
# 运行次数
max_steps = 3000

# 定义配置文件
config = projector.ProjectorConfig()
embed = config.embeddings.add()
embed.tensor_name = embedding.name
embed.metadata_path = PROJECTOR_META_FILENAME   #labels   相对于FileWriter目录的路径
embed.sprite.image_path = SPRITE_PNG_IMAGE      #images   相对于FileWriter目录的路径
# 切分图片
embed.sprite.single_image_dim.extend([28,28])

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    with tf.summary.FileWriter(PROJECTOR_DIR,sess.graph) as logWriter:
        projector.visualize_embeddings(logWriter, config)       #需要Metadata，embedding,config,saver导出为model.ckpt
        for i in range(max_steps):
            batch_xs,batch_ys = mnist.train.next_batch(batch_size)
            run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
            run_metadata = tf.RunMetadata()
            summary,_  = sess.run([merged, train_step], feed_dict={x:batch_xs,y:batch_ys}, options=run_options, run_metadata=run_metadata)
            logWriter.add_run_metadata(run_metadata,'step%03d' % i,i)
            logWriter.add_summary(summary, i)
            if i%100 == 0:
                acc = sess.run(accuracy, feed_dict={x:mnist.test.images, y:mnist.test.labels})
                print("Iter " + str(i) + ", Testing Accuracy= " + str(acc))
    saver.save(sess,os.path.join(PROJECTOR_DIR,"model.ckpt"),max_steps)

Iter 0, Testing Accuracy= 0.1955
Iter 100, Testing Accuracy= 0.9089
Iter 200, Testing Accuracy= 0.9316
Iter 300, Testing Accuracy= 0.9379
Iter 400, Testing Accuracy= 0.9478
Iter 500, Testing Accuracy= 0.9539
Iter 600, Testing Accuracy= 0.9562
Iter 700, Testing Accuracy= 0.9623
Iter 800, Testing Accuracy= 0.9648
Iter 900, Testing Accuracy= 0.9663
Iter 1000, Testing Accuracy= 0.9673
Iter 1100, Testing Accuracy= 0.968
Iter 1200, Testing Accuracy= 0.9712
Iter 1300, Testing Accuracy= 0.9705
Iter 1400, Testing Accuracy= 0.9713
Iter 1500, Testing Accuracy= 0.9717
Iter 1600, Testing Accuracy= 0.9737
Iter 1700, Testing Accuracy= 0.9737
Iter 1800, Testing Accuracy= 0.9747
Iter 1900, Testing Accuracy= 0.975
Iter 2000, Testing Accuracy= 0.9753
Iter 2100, Testing Accuracy= 0.9756
Iter 2200, Testing Accuracy= 0.9755
Iter 2300, Testing Accuracy= 0.9764
Iter 2400, Testing Accuracy= 0.9779
Iter 2500, Testing Accuracy= 0.9763
Iter 2600, Testing Accuracy= 0.9769
Iter 2700, Testing Accuracy= 0.9769
Iter 2